In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pulp import *
import pulp 
from tabulate import tabulate

In [ ]:
# Test pulp installation
pulp.pulpTestAll()

sssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss...........................................................sssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss....................

	 Test that logic put in place for deprecation handling of indexs works
	 Testing 'indexs' param continues to work for LpVariable.dicts
	 Testing 'indexs' param continues to work for LpVariable.matrix
	 Testing 'indices' argument works in LpVariable.dicts
	 Testing 'indices' param continues to work for LpVariable.matrix
	 Testing invalid status
	 Testing continuous LP solution - export dict
	 Testing export dict for LP
	 Testing export dict MIP
	 Testing maximize continuous LP solution
	 Testing continuous LP solution - export JSON
	 Testing continuous LP solution - export solver dict
	 Testing continuous LP solution - export solver JSON
	 Testing reading MPS files - binary variable, no constraint names
	 Testing reading MPS files - integer variable
	 Testing reading MPS files - maximize
	 Testing reading MPS files - noname
	 Testing invalid var names
	 Testing makeDict general behavior
	 Testing makeDict default value behavior
	 Testing measuring optimization time
	 Testing that `read

..

	 Testing makeDict general behavior
	 Testing makeDict default value behavior
	 Testing measuring optimization time


.....................................sssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss
----------------------------------------------------------------------
Ran 1003 tests in 10.935s

OK (skipped=885)


	 Testing that `readsol` can parse CPLEX mipopt solution
	 Testing the availability of the function pulpTestAll
	 Testing zero subtraction
	 Testing inconsistent lp solution
	 Testing continuous LP solution
	 Testing maximize continuous LP solution
	 Testing unbounded continuous LP solution
	 Testing Long Names
	 Testing repeated Names
	 Testing zero constraint
	 Testing zero objective
	 Testing LpVariable (not LpAffineExpression) objective
	 Testing Long lines in LP
	 Testing LpAffineExpression divide
	 Testing MIP solution
	 Testing MIP solution with floats in objective
	 Testing Initial value in MIP solution
	 Testing fixing value in MIP solution
	 Testing MIP relaxation
	 Testing feasibility problem (no objective)
	 Testing an infeasible problem
	 Testing an integer infeasible problem
	 Testing another integer infeasible problem
	 Testing column based modelling
	 Testing dual variables and slacks reporting
	 Testing fractional constraints
	 Testing elastic constraints (no change)
	

In [ ]:
def check_layers(name):
    layers = [
        "dense_1.weight_integer",
        "dense_1.bias_integer",
        "dense_2.weight_integer",
        "dense_2.bias_integer",
        "dense_3.weight_integer",
        "dense_3.bias_integer",
        "dense_4.weight_integer",
        "dense_4.bias_integer",
    ]
    for layer in layers:
        if name == layer:
            return True
    return False

def count_hawq_nonzero_weights(model):
    nonzero = total = 0
    layer_count_alive = {}
    layer_count_total = {}
    for name, p in model.named_buffers():
        if check_layers(name):
            tensor = p.data.cpu().numpy()
            nz_count = np.count_nonzero(tensor)
            total_params = np.prod(tensor.shape)
            layer_count_alive.update({name: nz_count})
            layer_count_total.update({name: total_params})
            nonzero += nz_count
            total += total_params
            logging.info(
                f"{name:20} | nonzeros = {nz_count:7} / {total_params:7} ({100 * nz_count / total_params:6.2f}%) | total_pruned = {total_params - nz_count :7} | shape = {tensor.shape}"
            )
    logging.info(
        f"alive: {nonzero}, pruned : {total - nonzero}, total: {total}, Compression rate : {total/nonzero:10.2f}x  ({100 * (total-nonzero) / total:6.2f}% pruned)"
    )
    return (
        nonzero,
        total,
        layer_count_alive,
        layer_count_total,
        (100 * (total - nonzero) / total),
    )


def calc_hawq_bops(model, input_data_precision=32):
    last_bit_width = input_data_precision
    alive, total, l_alive, l_total, pruned = count_hawq_nonzero_weights(model)
    b_w = model.dense_1.bias_bit if hasattr(model, "dense_1") else 32
    total_BOPS = 0
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear) or isinstance(module, QuantLinear):
            b_a = last_bit_width
            # b_w = module.quant_weight_bit_width #Dont think this is a property I can access sadly, going with precision as given set in model
            n = module.in_features
            m = module.out_features
            total = l_total[name + ".weight_integer"] + l_total[name + ".bias_integer"]
            alive = l_alive[name + ".weight_integer"] + l_alive[name + ".bias_integer"]
            p = 1 - ((total - alive) / total)  # fraction of layer remaining
            # assuming b_a is the output bitwidth of the last layer
            # module_BOPS = m*n*p*(b_a*b_w + b_a + b_w + math.log2(n))
            module_BOPS = m * n * (p * b_a * b_w + b_a + b_w + math.log2(n))
            logging.info(
                "{} BOPS: {} = {}*{}({}*{}*{} + {} + {} + {})".format(
                    name, module_BOPS, m, n, p, b_a, b_w, b_a, b_w, math.log2(n)
                )
            )
            last_bit_width = b_w
            total_BOPS += module_BOPS
    logging.info("Total BOPS: {}".format(total_BOPS))
    return total_BOPS, pruned